In [ ]:
import json
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R

# === STEP 1: Load IMU Raw JSONL Data ===
raw_path = "logs/imu_raw_2025_04_19.jsonl"
with open(raw_path, "r") as f:
    raw_data = [json.loads(line) for line in f]

from ahrs.filters import Madgwick
from ahrs.common.orientation import acc2q

# Create one filter per IMU to maintain state
madgwick_filters = {
    "left_upper": Madgwick(),
    "left_forearm": Madgwick(),
    "right_upper": Madgwick(),
    "right_forearm": Madgwick()
}

# Store previous quaternions
imu_quat_state = {
    "left_upper": np.array([1.0, 0.0, 0.0, 0.0]),
    "left_forearm": np.array([1.0, 0.0, 0.0, 0.0]),
    "right_upper": np.array([1.0, 0.0, 0.0, 0.0]),
    "right_forearm": np.array([1.0, 0.0, 0.0, 0.0])
}

def simulate_quaternion(accel, gyro, imu_id, dt=0.05):
    """
    Computes quaternion orientation using Madgwick filter.
    accel: [ax, ay, az]
    gyro: [gx, gy, gz] in rad/s
    imu_id: one of ["left_upper", "left_forearm", ...]
    """
    a = np.array(accel)
    g = np.radians(np.array(gyro))  # convert deg/s to rad/s

    filt = madgwick_filters[imu_id]
    q_prev = imu_quat_state[imu_id]

    q_new = filt.updateIMU(q_prev, g, a)
    if q_new is not None:
        imu_quat_state[imu_id] = q_new
        return q_new.tolist()
    else:
        return q_prev.tolist()  # fallback

# === STEP 3: Relative Angle from Two Quaternions ===
def relative_angle(q1, q2):
    r1 = R.from_quat([q1[1], q1[2], q1[3], q1[0]])
    r2 = R.from_quat([q2[1], q2[2], q2[3], q2[0]])
    rel_rot = r2 * r1.inv()
    return rel_rot.magnitude() * (180 / np.pi)

# === STEP 4: Process Frame-by-Frame ===
results = []

for frame in raw_data:
    try:
        # Simulate orientation from accel (placeholder for real fusion)
        q_lu = simulate_quaternion(frame["left_upper"]["accel"])
        q_lu = simulate_quaternion(
            frame["left_upper"]["accel"],
            frame["left_upper"]["gyro"],
            "left_upper"
        )
        q_lf = simulate_quaternion(
            frame["left_forearm"]["accel"],
            frame["left_forearm"]["gyro"],
            "left_forearm"
        )
        q_ru = simulate_quaternion(
            frame["right_upper"]["accel"],
            frame["right_upper"]["gyro"],
            "right_upper"
        )
        q_rf = simulate_quaternion(
            frame["right_forearm"]["accel"],
            frame["right_forearm"]["gyro"],
            "right_forearm"
        )

        
        q_torso = [1, 0, 0, 0]  # Assume torso neutral for now

        angles = {
            "left_shoulder_angle": relative_angle(q_torso, q_lu),
            "right_shoulder_angle": relative_angle(q_torso, q_ru),
            "left_elbow_angle": relative_angle(q_lu, q_lf),
            "right_elbow_angle": relative_angle(q_ru, q_rf),
            "timestamp": frame["timestamp"]
        }

        results.append(angles)

    except Exception as e:
        print("Error in frame:", e)

df = pd.DataFrame(results)

# === STEP 5: Estimate "up" or "down" phase from elbow angle velocity ===
df["elbow_velocity"] = df["left_elbow_angle"].diff() / df["timestamp"].diff()

def classify_phase(v, threshold=0.05):
    if v < -threshold:
        return "b_correct_up"
    elif v > threshold:
        return "b_correct_down"
    else:
        return "b_correct_pause"

df["label"] = df["elbow_velocity"].apply(classify_phase)

# === STEP 6: Final Output Dataset ===
final_df = df[[
    "left_shoulder_angle",
    "right_shoulder_angle",
    "left_elbow_angle",
    "right_elbow_angle",
    "label"
]]

final_df.to_csv("imu_angles_labeled.csv", index=False)
print("✅ Saved imu_angles_labeled.csv")
